<a href="https://colab.research.google.com/github/armandossrecife/mysentimentanalysis/blob/main/my_automatic_inspection_issues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/armandossrecife/mychatbot/issue_7/issues_ground_truth.json

--2024-07-06 16:36:06--  https://raw.githubusercontent.com/armandossrecife/mychatbot/issue_7/issues_ground_truth.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1593692 (1.5M) [text/plain]
Saving to: ‘issues_ground_truth.json’

issues_ground_truth 100%[===================>]   1.52M  --.-KB/s    in 0.07s   

2024-07-06 16:36:06 (23.0 MB/s) - ‘issues_ground_truth.json’ saved [1593692/1593692]



In [9]:
!wget https://raw.githubusercontent.com/Technical-Debt-Large-Scale/my_validation/main/cassandra_issues_inspected_merged.xlsx

--2024-07-06 17:13:10--  https://raw.githubusercontent.com/Technical-Debt-Large-Scale/my_validation/main/cassandra_issues_inspected_merged.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 573631 (560K) [application/octet-stream]
Saving to: ‘cassandra_issues_inspected_merged.xlsx’

cassandra_issues_in 100%[===================>] 560.19K  --.-KB/s    in 0.05s   

2024-07-06 17:13:10 (11.6 MB/s) - ‘cassandra_issues_inspected_merged.xlsx’ saved [573631/573631]



In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [42]:
# prompt: how to convert json file to pandas dataframe?
import pandas as pd
import json
from datasets import load_dataset

with open('issues_ground_truth.json') as f:
    data = json.load(f)

df = pd.json_normalize(data)
df_issues_ground_truth = pd.DataFrame(df)
df_issues_ground_truth.head()

,issue_key,issue_type,summary,description,comments,architectural_impact_manual
0,CASSANDRA-11272,Bug,NullPointerException (NPE) during bootstrap st...,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES
1,CASSANDRA-1321,Improvement,loadSchemaFromYaml should push migrations to c...,None,"['announces the last migration, which ends up ...",YES
2,CASSANDRA-1641,Bug,auto-guessed memtable sizes are too high,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES
3,CASSANDRA-2296,Bug,"Scrub resulting in ""bloom filter claims to be ...",Doing a scrub on a node which I upgraded from ...,"[""With debug logging turned on it looks like t...",YES
4,CASSANDRA-3117,Bug,StorageServiceMBean is missing a getCompaction...,"Without a getter, you can assign a new value b...","['+1', 'committed.', 'Integrated in Cassandra-...",YES


In [43]:
df_issues_ground_truth.columns

Index(['issue_key', 'issue_type', 'summary', 'description', 'comments',
       'architectural_impact_manual'],
      dtype='object')

In [44]:
# prompt: how to convert .xls file to pandas dataframe?

df_cassandra_issues_inspected_merged = pd.read_excel('cassandra_issues_inspected_merged.xlsx')
df_cassandra_issues_inspected_merged.head()


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact_manual,architectural_impact,TP,TN,FP,FN
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,YES,1,0,0,0
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,YES,1,0,0,0
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,YES,1,0,0,0
3,CASSANDRA-2296,"Scrub resulting in ""bloom filter claims to be ...",Bug,Resolved,Normal,Doing a scrub on a node which I upgraded from ...,"[""With debug logging turned on it looks like t...",YES,YES,1,0,0,0
4,CASSANDRA-3117,StorageServiceMBean is missing a getCompaction...,Bug,Resolved,Low,"Without a getter, you can assign a new value b...","['+1', 'committed.', 'Integrated in Cassandra-...",YES,YES,1,0,0,0


In [45]:
df_cassandra_issues_inspected_merged.columns

Index(['issue_key', 'summary', 'issue_type', 'issue_status', 'issue_priority',
       'description', 'comments', 'architectural_impact_manual',
       'architectural_impact', 'TP', 'TN', 'FP', 'FN'],
      dtype='object')

In [47]:
colunas_manual = ['issue_key', 'summary', 'issue_type', 'issue_status', 'issue_priority',
       'description', 'comments', 'architectural_impact_manual']
df_cassandra_issues_inspected_manual = df_cassandra_issues_inspected_merged[colunas_manual]
df_cassandra_issues_inspected_manual.head(3)

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact_manual
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES


In [48]:
colunas_automatica = ['issue_key', 'summary', 'issue_type', 'issue_status', 'issue_priority',
       'description', 'comments', 'architectural_impact']
df_cassandra_issues_inspected_automatica = df_cassandra_issues_inspected_merged[colunas_automatica]
df_cassandra_issues_inspected_automatica.head(3)

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES


In [50]:
def convert_comment_to_text(comments_string):
  try:
    comments_string = comments_string.split(": ")[-1]  # Assuming the colon and space separate the key-value pair
    comments_list = comments_string.split(",")
    comments = ""
    for comment in comments_list:
        comment = comment.replace("['", "")
        comment = comment.replace("']", "")
        comment = comment.strip()
        comments += comment + " "
    return comments
  except:
    return ""

def convert_inspection_to_binary(inspection):
  if inspection == 'YES':
    return 1
  else:
    return 0

def convert_inspection_to_sentiment(inspection):
  if inspection == 'YES':
    return 'positive'
  else:
    return 'negative'

In [51]:
df_cassandra_issues_inspected_manual['comments_text'] = df_cassandra_issues_inspected_manual['comments'].apply(convert_comment_to_text)
df_cassandra_issues_inspected_manual['label'] = df_cassandra_issues_inspected_manual['architectural_impact_manual'].apply(convert_inspection_to_binary)
df_cassandra_issues_inspected_manual['label_text'] = df_cassandra_issues_inspected_manual['architectural_impact_manual'].apply(convert_inspection_to_binary)

<ipython-input-51-3e2e36717764>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected_manual['comments_text'] = df_cassandra_issues_inspected_manual['comments'].apply(convert_comment_to_text)
<ipython-input-51-3e2e36717764>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected_manual['label'] = df_cassandra_issues_inspected_manual['architectural_impact_manual'].apply(convert_inspection_to_binary)
<ipython-input-51-3e2e36717764>:3: SettingWithCopyWarning: 
A 

In [52]:
df_cassandra_issues_inspected_manual.head(3)

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact_manual,comments_text,label,label_text
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,patch is at the end of [ifesdjeen/11272-2.2|ht...,1,1
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,announces the last migration which ends up pus...,1,1
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,"[""I'd like to introduce a dependency on number...",1,1


In [54]:
df_cassandra_issues_inspected_automatica['comments_text'] = df_cassandra_issues_inspected_automatica['comments'].apply(convert_comment_to_text)
df_cassandra_issues_inspected_automatica['label'] = df_cassandra_issues_inspected_automatica['architectural_impact'].apply(convert_inspection_to_binary)
df_cassandra_issues_inspected_automatica['label_text'] = df_cassandra_issues_inspected_automatica['architectural_impact'].apply(convert_inspection_to_binary)

<ipython-input-54-0df86bec3cb5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected_automatica['comments_text'] = df_cassandra_issues_inspected_automatica['comments'].apply(convert_comment_to_text)
<ipython-input-54-0df86bec3cb5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected_automatica['label'] = df_cassandra_issues_inspected_automatica['architectural_impact'].apply(convert_inspection_to_binary)
<ipython-input-54-0df86bec3cb5>:3: SettingWithCopyWar

In [55]:
df_cassandra_issues_inspected_automatica.head()

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,patch is at the end of [ifesdjeen/11272-2.2|ht...,1,1
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,announces the last migration which ends up pus...,1,1
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,"[""I'd like to introduce a dependency on number...",1,1
3,CASSANDRA-2296,"Scrub resulting in ""bloom filter claims to be ...",Bug,Resolved,Normal,Doing a scrub on a node which I upgraded from ...,"[""With debug logging turned on it looks like t...",YES,Keys must be written in ascending order.\n ...,1,1
4,CASSANDRA-3117,StorageServiceMBean is missing a getCompaction...,Bug,Resolved,Low,"Without a getter, you can assign a new value b...","['+1', 'committed.', 'Integrated in Cassandra-...",YES,\n* /cassandra/branches/cassandra-0.8/src/java...,1,1


In [62]:
df_cassandra_issues_inspected_automatica.columns

Index(['issue_key', 'summary', 'issue_type', 'issue_status', 'issue_priority',
       'description', 'comments', 'architectural_impact', 'comments_text',
       'label', 'label_text'],
      dtype='object')

In [56]:
df_data_test = df_cassandra_issues_inspected_automatica
df_data_test.head()

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,patch is at the end of [ifesdjeen/11272-2.2|ht...,1,1
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,announces the last migration which ends up pus...,1,1
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,"[""I'd like to introduce a dependency on number...",1,1
3,CASSANDRA-2296,"Scrub resulting in ""bloom filter claims to be ...",Bug,Resolved,Normal,Doing a scrub on a node which I upgraded from ...,"[""With debug logging turned on it looks like t...",YES,Keys must be written in ascending order.\n ...,1,1
4,CASSANDRA-3117,StorageServiceMBean is missing a getCompaction...,Bug,Resolved,Low,"Without a getter, you can assign a new value b...","['+1', 'committed.', 'Integrated in Cassandra-...",YES,\n* /cassandra/branches/cassandra-0.8/src/java...,1,1


In [63]:
#architectural_impact
df_cassandra_issues_inspected_manual.columns

Index(['issue_key', 'summary', 'issue_type', 'issue_status', 'issue_priority',
       'description', 'comments', 'architectural_impact_manual',
       'comments_text', 'label', 'label_text'],
      dtype='object')

In [64]:
df_cassandra_issues_inspected_manual['architectural_impact'] = df_cassandra_issues_inspected_manual['architectural_impact_manual']

<ipython-input-64-4577b7183cee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected_manual['architectural_impact'] = df_cassandra_issues_inspected_manual['architectural_impact_manual']


In [65]:
my_columns = ['issue_key', 'summary', 'issue_type', 'issue_status', 'issue_priority',
       'description', 'comments', 'architectural_impact', 'comments_text',
       'label', 'label_text']
df_cassandra_issues_inspected_manual = df_cassandra_issues_inspected_manual[my_columns]
df_cassandra_issues_inspected_manual.head(3)

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,patch is at the end of [ifesdjeen/11272-2.2|ht...,1,1
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,announces the last migration which ends up pus...,1,1
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,"[""I'd like to introduce a dependency on number...",1,1


In [66]:
df_data_train = df_cassandra_issues_inspected_manual
df_data_train.head()

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,patch is at the end of [ifesdjeen/11272-2.2|ht...,1,1
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,announces the last migration which ends up pus...,1,1
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,"[""I'd like to introduce a dependency on number...",1,1
3,CASSANDRA-2296,"Scrub resulting in ""bloom filter claims to be ...",Bug,Resolved,Normal,Doing a scrub on a node which I upgraded from ...,"[""With debug logging turned on it looks like t...",YES,Keys must be written in ascending order.\n ...,1,1
4,CASSANDRA-3117,StorageServiceMBean is missing a getCompaction...,Bug,Resolved,Low,"Without a getter, you can assign a new value b...","['+1', 'committed.', 'Integrated in Cassandra-...",YES,\n* /cassandra/branches/cassandra-0.8/src/java...,1,1


In [69]:
# prompt: how to convert pandas dataframe to json file?
import json

def convert_to_json(df):
    try:
      json_data = df.to_json(orient='records')
      return json.loads(json_data)
    except Exception as e:
      raise ValueError(e)

def write_json_to_file(json_data, filename):
    try:
      with open(filename, 'w') as f:
        json.dump(json_data, f)
    except Exception as e:
      raise ValueError(e)

def convert_dataframe_to_json_file(df, filename):
  try:
    json_string_data = convert_to_json(df)
    write_json_to_file(json_string_data, filename)
  except Exception as e:
    raise ValueError(e)

# Convert the DataFrame to a JSON string
convert_dataframe_to_json_file(df=df_data_train, filename='train.json')
convert_dataframe_to_json_file(df=df_data_test, filename='test.json')

In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
!pip install accelerate -U

In [4]:
from transformers import DistilBertForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer  # Import from transformers
import torch

In [5]:
# Define model name and task
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
task = "sentiment-analysis"

In [6]:
# Load pre-trained DistilBERT model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [70]:
MY_HUGGING_FACE_DATASET = "armandoufpi/cassandraissuesgroundtruth"

In [71]:
# Load the IMDB movie review dataset
train_data = load_dataset(MY_HUGGING_FACE_DATASET, split="train")
test_data = load_dataset(MY_HUGGING_FACE_DATASET, split="test")

Generating train split:   0%|          | 0/226 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/226 [00:00<?, ? examples/s]

In [72]:
!mkdir results

In [73]:
!ls -l

total 5988
-rw-r--r-- 1 root root  573631 Jul  6 17:13 cassandra_issues_inspected_merged.xlsx
-rw-r--r-- 1 root root 1593692 Jul  6 16:36 issues_ground_truth.json
drwxr-xr-x 2 root root    4096 Jul  6 18:11 results
drwxr-xr-x 1 root root    4096 Jul  3 13:21 sample_data
-rw-r--r-- 1 root root 1971696 Jul  6 18:06 test.json
-rw-r--r-- 1 root root 1971676 Jul  6 18:06 train.json


In [ ]:
training_args = TrainingArguments(
    output_dir="results",  # Fixed typo (removed extra space)
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Assuming you meant "size" here
    learning_rate=2e-5,
)

In [ ]:
# Create trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics="accuracy",
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
new_review = "This movie was amazing! Highly recommend."
inputs = tokenizer(new_review, padding="max_length", truncation=True, return_tensors="pt")

# Move the model and input to GPU if available
if torch.cuda.is_available():
  model.to('cuda')
  inputs.to('cuda')

with torch.no_grad():
  outputs = model(**inputs)
  predictions = torch.argmax(outputs.logits, dim=-1)

# Print the predicted sentiment
if predictions == 1:
  print("Sentiment: Positive")
else:
  print("Sentiment: Negative")